In [1]:
import pandas as pd
testing_frame = pd.read_parquet("/kaggle/input/dactyl-adversarial/finetuned-texts.parquet")

testing_frame = testing_frame[testing_frame.split == "testing"]

In [2]:
# Use a pipeline as a high-level helper
from transformers import pipeline
from tqdm import tqdm
pipe = pipeline("text-classification", model="MayZhou/e5-small-lora-ai-generated-detector")

def data():
    for text in testing_frame["text"].values:
        yield text
y_pred = list()
for result in tqdm(pipe(data(),batch_size=16, max_length=512,truncation=True, return_all_scores=True), total=len(testing_frame)):
    for output in result:
        if output["label"] == "LABEL_1":
            y_pred.append(output["score"])



config.json:   0%|          | 0.00/643 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

Device set to use cuda:0
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
100%|██████████| 5516/5516 [00:29<00:00, 188.09it/s]


In [3]:
testing_frame["e5lora_pred"] = y_pred
testing_frame[["id","e5lora_pred","target","model","domain"]].to_csv(f"e5lora-results.csv",index=False)
